In [34]:
import numpy as np
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import random

from numpy.distutils.command.egg_info import egg_info

In [2]:
pts = np.array([np.random.uniform([-20, -20, 0], [20, 20, 5]) for _ in range(50)])
G = nx.Graph()
for i in range(len(pts)):
    G.add_node(i, pos=tuple(pts[i]))

for i in range(len(pts)):
        for j in range(i+1, len(pts)):
            if np.random.rand() < 0.3:
                distance = np.linalg.norm(pts[i] - pts[j])
                G.add_edge(i, j, distance=distance)

In [90]:
A = nx.adjacency_matrix(G, weight='distance')

In [3]:
elevations = [G.nodes[i]['pos'][2] for i in G.nodes]
treatment_idxs = np.argsort(elevations)[:5]


In [4]:
for i in G.nodes:
    if i in treatment_idxs:
        G.nodes[i]['treatment'] = 1
    else:
        G.nodes[i]['treatment'] = 0

In [5]:
treatment_nodes = [i for i in G.nodes if G.nodes[i]['treatment'] == 1]
source_nodes = [i for i in G.nodes if G.nodes[i]['treatment'] == 0]

In [6]:
print(f"# Source: {len(source_nodes)}; # treatment: {len(treatment_nodes)}; # Components = {nx.number_connected_components(G)}")

Path = {}
NLinks = {}
L = {}

# Source: 45; # treatment: 5; # Components = 1


In [7]:
for i in source_nodes:
    for j in treatment_nodes:
        path = nx.shortest_path(G, source=i, target=j, weight='distance')
        Path[i, j] = path
        NLinks[i, j] = len(path)
        L[i, j] = nx.path_weight(G, Path[i, j], weight='distance')

In [12]:
print(f"For the path from source node 1 to treatment node 5: path = {Path[1, 5]}; NLinks = {NLinks[1, 5]}; L = {L[1, 5]}")

For the path from source node 1 to treatment node 5: path = [1, 31, 5]; NLinks = 3; L = 19.087024306888647


In [76]:
LE = {e:G.edges[e]['distance'] for e in G.edges}
EL = {v: G.nodes[v]['pos'][2] for v in G.nodes}
D = [0.2, 0.25, 0.3, 0.35, 0.40, 0.45]

In [20]:
mu = 1000
sigma = 200
SR = {}
CAP = {}

for node in source_nodes:
    flow = np.random.normal(mu, sigma)
    G.nodes[node]['production'] = flow
    SR[node] = flow

total_flow = sum(SR.values())

for node in treatment_nodes:
    capacity = 1.25 * (total_flow / len(treatment_nodes))
    G.nodes[node]['capacity'] = capacity
    CAP[node] = capacity

Vmin = 0.6
Vmax = 3


In [105]:
CE = 25
CB = 6
TR = 44000
TRFlow = 100

PICost = 30
PF = {'0.05': 8.7, '0.06': 9.5, '0.08': 11,
                       '0.1': 12.6, '0.15': 43.5, '0.2': 141,
                       '0.25': 151, '0.3': 161, '0.35': 180,
                       '0.4': 190, '0.45': 200}

CT = 10000
M = 1e10

In [53]:
m = gp.Model()

In [108]:
# DECISION VARIABLES

x = m.addVars(Path.keys(), vtype=GRB.BINARY, name=[f'x_({i},{j})' for i, j in Path.keys()])
y = m.addVars(treatment_nodes, vtype=GRB.BINARY, name=[f'y_{j}' for j in treatment_nodes])
z = m.addVars(G.edges, vtype=GRB.BINARY, name=[f'z_{e}' for e in G.edges])

d_es_names = []
for e in G.edges:
    for s in D:
        d_es_names.append(f"d_{s}_{e}")
d_es = m.addVars(D, G.edges, vtype=GRB.BINARY, name=d_es_names)

r = m.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb=0.0, ub=SR, name=[f'r_{v}' for v in G.nodes])
Q = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb=0.0, name=[f'q_{e}' for e in G.edges])

e_in = m.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb=0.0, name=[f'e_in_{v}' for v in G.nodes])
e_out = m.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb=0.0, name=[f'e_out_{v}' for v in G.nodes])
h = m.addVars(G.nodes, vtype=GRB.CONTINUOUS, name=[f'h_{v}' for v in G.nodes])
m.update()

In [109]:
# OBJECTIVE EXPR 1: TREATMENT COSTS

treat_cost = gp.LinExpr()

for j in treatment_nodes:
    treat_cost.addTerms(TR, y[j])
    for i in source_nodes:
        treat_cost.addTerms(TRFlow * SR[i], x[i, j])

In [ ]:
# OBJECTIVE EXPR 2: EXCAVATION COSTS
excav_cost = gp.LinExpr()

In [ ]:
# OBJECTIVE EXPR 3: BEDDING COSTS
bed_cost = gp.LinExpr()

In [106]:
# OBJECTIVE EXPR 4: RECOURSE TRUCKING

rec_cost = gp.LinExpr()
for i in source_nodes:
    rec_cost.addTerms(CT, r[i])

In [110]:
# CONSTRAINTS

# TREATMENT CAPACITY
treat_cap = m.addConstrs(gp.quicksum(SR[i] * x[i, j] - r[i] for i in source_nodes)<= CAP[j]*y[j] for j in treatment_nodes)

#  NODE ASSIGNMENT
node_assign = m.addConstrs(gp.quicksum(x[i, j] for j in treatment_nodes) == 1 for i in source_nodes)

In [ ]:
# EDGE ACTIVATION

# TODO: Need edge-wise path

In [114]:
# PIPE SIZING

pipe_sizing = m.addConstrs(gp.quicksum(d_es[s, *e] for s in D) == 1 for e in G.edges)

In [118]:
# HEIGHT DIFFERENCE
height = m.addConstrs(h[i] == e_out[i] - e_in[i] for i in source_nodes)

# MINUMUM SLOPE

# MAXIMUM SLOPE

In [117]:
# FLOW VELOCITY LIMIT

# TODO: D structure needs to be fixed, and d_es should be renamed

flow_vel = m.addConstrs(Q[e] <= gp.quicksum((np.pi / 8) * (D[s]**2) * (d_es[D[s], *e]) for s in range(len(D))) for e in G.edges)

In [ ]:
# MANNING'S EQUATION


In [125]:
epr = gp.MQuadExpr
for u, v in G.edges:
    m.addQConstr(LE[u, v] * gp.quicksum(d_es[s, u, v] * ((Q[u, v] / (11.9879 * (s**(8/3))))**2) for s in D)  <= e_in[u] - e_in[v] + h[u])


GurobiError: Invalid argument to QuadExpr multiplication

In [129]:
for u, v in G.edges:
    expr = gp.QuadExpr()
    for s in D:
        expr.addTerms(LE[u, v] * d_es[s, u, v], Q[u, v] ** 2 / (11.9879 * s ** (8/3)), Q[u, v])

    m.addQConstr(expr <= e_in[u] - e_in[v] + h[u])

TypeError: object of type 'gurobipy.LinExpr' has no len()

In [123]:
# NO HEIGHT GAIN

# PIPES UNDERGROUND
underground = m.addConstrs(e_in[i] <= EL[i] for i in source_nodes)